In [10]:
import sqlalchemy
from sqlalchemy import create_engine
from urllib.parse import quote 
from datetime import datetime
from sqlalchemy.exc import OperationalError
import pandas as pd
from urllib.parse import quote
from dateutil.relativedelta import relativedelta
import pyodbc
import tqdm
import openpyxl
from openpyxl import load_workbook
import os
from tqdm import tqdm

In [11]:
### Run Query in chunks

#***Yield chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

#*** Remove brackets from list
def remove_list_brackets(list_):
    str_no_list = str(list_)
    str_no_list = str_no_list.replace('[','(').replace(']',')')
    return str_no_list

In [12]:
#**** Engine
#**** Engine
engine_=sqlalchemy.create_engine('mssql+pyodbc://SAL_User01:%s@172.16.22.25:1433/FINCORE_BRIDGE_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@123'))


In [13]:
pwd

'C:\\Users\\LTIM_10737710'

In [14]:
pans_list = pd.read_excel(r"\\172.20.1.68\shared\abhinav\pans_list_pak_related.xlsx")

In [15]:
pans_list.head(2)

,pan
0,AAICC8794L
1,CZXPA0145E


In [16]:
len(pans_list)

4706

In [17]:

def query_run(query,engine_,path_to_file, file_name,mode = 'a'):
    '''
    **query**: ('str') Enter the query 
    **engine_** : ('str') the function creates a connection with postgres server 172.16.22.15:5432/postgres as default,
              other engines can be specified here as well
    **mode**: ('str') mode is append by default, can be changed to simply write as well
    **path_to_file**: ('str') path where the file to be saved
    **filename**: ('str') name of the file, defaults get saved as csv
    '''
    if mode == 'a':
        try:
            print('--Engine Running--')
            engine_ = engine_
            conn = engine_.connect()
            results = pd.read_sql(query,conn)
            conn.close()
            engine_.dispose()
            print('--Engine Closing--')
            print('Results_appending')
            file_name = f'{file_name}.csv'
            path = os.path.join(path_to_file,file_name)
            results.to_csv(path, index = False, mode = mode, header= not os.path.exists(path))
        except OperationalError as oe:
            time.sleep(300)
            print('Reconnecting with Server')
            print('--Engine Running--')
            engine_ = engine_
            conn = engine_.connect()
            results = pd.read_sql(query,conn)
            conn.close()
            engine_.dispose()
            print('--Engine Closing--')
            print('Results_appending')
            filename = f'{file_name}.csv'
            path = os.path.join(path_to_file,file_name)
            results.to_csv(path, index = False, mode = mode, header= not os.path.exists(path))
    else:
        print('--Engine Running--')
        engine_ = engine_
        conn = engine_.connect()
        results = pd.read_sql(query,engine_)
        conn.close()
        engine_.dispose()
        print('--Engine Closing--')
        print('Results_appending')
        file_name = f'{file_name}.csv'
        path = os.path.join(path_to_file,file_name)
        results.to_csv(path, index = False, mode = mode)

In [18]:
#**** Run query in chunks
#reid_acc_ts_final_list = list(reid_acc_ts_final['reid_acc'].unique())
str_data_pans = list(pans_list['pan'].unique())
#str_data_pans

In [ ]:

for chunk in tqdm(list(chunks(str_data_pans,50))):
    bracket = remove_list_brackets(chunk)
    query = f'''  with cte2 as (
        select batchid,pd.reportid, pd.ucic, PAN, concat(firstname,' ', middlename,' ',lastname) personname, cast(occupationid as nvarchar) as occupation,
        concat(pd.primaryAddressLine1,' | ',pd.primaryLocality) as address,primaryPinCode as person_Pincode , pd.nationality , pd.telephoneNumber as telephone, pd.mobileNumber as mobile, email from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_PersonDetail] (nolock) pd 
        where  migrated_flag is null  and pd.pan in {bracket} --enter PAN here   
        union
        select batchid, ed.reportid, ed.ucic, PAN, ed.name as personname ,linesOfBusiness as occupation, 
        concat(ed.addressLine1,' | ',ed.locality) as address, pinCode as person_Pincode ,ed.regCountry as nationality, telephoneNumber, mobileNumber, emailId
        from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_EntityDetail] (nolock) ed where ed.pan in {bracket} --enter PAN here
    ),
    cte3 as ( 
        select reportid, ucic  from( select reportid, ucic, count(distinct UCIC) pan_count from cte2 group by reportid, ucic)z where pan_count =1),
    cte4 as (
        select * from cte2 where concat(reportid, ucic) in (select concat(reportid, ucic) from cte3))

    select distinct  coalesce([oldREId],[newREId]) as fiureid,trim(upper(re.name)) as entityname, concat('A-',ad.accountNumber)accountnumber,concat('ID-',cast([Fiu_Batch_Id] as varchar)) batchid,r.Report_Id as reportsernum,
    cte4.pan,personname, address as commaddress,concat('M-',mobile)mobile,email,
    Report_Type as reporttype,  gosTextNarration groundsofsusp,[gosTag2] sourceofalert,addressline1 as branchaddress , ah.title as relationTypeId from cte4
    inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details] r on r.Report_Id =reportid AND REPORT_TYPE = 'STR'  and r.report_status='Pass' 

    inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on r.[Re_Id] = re.id_
    inner join [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_AccountDetail] ad on r.Report_Id=ad.reportid
    inner join [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_AccountPerson] ap on ap.accountNo=ad.accountNumber and ap.reportid=ad.reportId and cte4.reportid=ap.reportid and cte4.ucic=ap.ucic 
    left join [DATAHUB_DB].[FIUMetaHub].[finnet_AcHolderType] ah on ah.id_=ap.relationTypeId
    left join [DATAHUB_DB].[FIUMetaHub].[finnet_ReBranch] reb on ad.branchCode=reb.branchId and reb.reid=r.Re_Id
    left join [DATAHUB_DB].[FIUMetaHub].[finnet_Address] adr on reb.id_ = adr.parentid and adr.parentclass = 'REBranch' and re.active_=1
    left join [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_Gos] (nolock) gs on r.Report_Id=gs.reportId
    left join [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_KycSummary] kyc on r.Report_Id=kyc.reportId '''
    #print(query)
    #break
    engine_=sqlalchemy.create_engine('mssql+pyodbc://SAL_User01:%s@172.16.22.25:1433/FINCORE_BRIDGE_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@123'))
    path_to_file = 'C:/Users/LTIM_10737710/Downloads/'
    file_name = 'str_related_pans_dump_pak'
    query_run(query,engine_,path_to_file,file_name)

  0%|                                                                                           | 0/95 [00:00<?, ?it/s]

--Engine Running--


In [ ]:
rid_1 = pd.read_excel('reportids_j&k_branch_state_202503.xlsx',sheet_name='NTR')

In [ ]:
#**** Run query in chunks
#reid_acc_ts_final_list = list(reid_acc_ts_final['reid_acc'].unique())
str_data_pans = list(rid['report_id'].unique())

for chunk in tqdm(list(chunks(str_data_pans,1000))):
    bracket = remove_list_brackets(chunk)
    query = f'''  with cte1 as (
        select Re_Id ,pd.batchid,Fiu_Batch_Id, left(Fiu_Batch_Id,6) yr_mon,pd.reportid, r.REPORT_id , pd.ucic,trim(accountno) as accountnumber,PAN, concat(firstname,' ', middlename,' ',lastname) personname, cast(occupationid as nvarchar) as occupation,

        concat(pd.primaryAddressLine1,' | ',pd.primaryLocality) as address,primaryPinCode as person_Pincode , pd.nationality , pd.telephoneNumber as telephone, pd.mobileNumber as mobile, email, Report_Type,relationTypeId from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_PersonDetail] (nolock) pd 
        inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details]r on r.Report_Id =pd.reportid AND REPORT_TYPE = 'NTR' 
        AND LEFT(FIU_BATCH_ID,6)=202503 and r.report_id in {bracket}
        inner join [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_AccountPerson] ap on ap.UCIC=pd.UCIC and ap.reportid=pd.reportId 
        and r.Fingate_Migrated_Data = 'N'  and r.report_status ='Pass' and r.Is_Active_Flag=1 
    union
        select Re_ID,ed.batchid,Fiu_Batch_Id,left(Fiu_Batch_Id,6) yr_mon,ed.reportid, r.REPORT_id,ed.ucic, trim(accountno) as accountnumber,PAN, ed.name as personname ,linesOfBusiness as occupation, 
        concat(ed.addressLine1,' | ',ed.locality) as address, pinCode as person_Pincode ,ed.regCountry as nationality, telephoneNumber, mobileNumber, emailId,REPORT_TYPE,relationTypeId
        from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_EntityDetail] (nolock) ed 
        inner join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Cps_Report_Details]r on r.Report_Id =ed.reportid AND REPORT_TYPE = 'NTR'
        AND LEFT(FIU_BATCH_ID,6)=202503 and r.report_id in {bracket}
    
        inner join [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_AccountPerson] ap on ap.UCIC=ed.UCIC and ap.reportid=ed.reportId 
        and r.Fingate_Migrated_Data = 'N'  and r.report_status ='Pass' and r.Is_Active_Flag=1 ),
        cte2 as ( select reportid, ucic  from( select reportid, ucic, count(distinct pan) pan_count from cte1 group by reportid, ucic)z where pan_count =1),
        cte3 as (select * from cte1 where concat(reportid, ucic) in (select concat(reportid, ucic) from cte2))
        ,

        cte_trans as (select distinct relationTypeId,re_id,t.[reportid],credit as [credit_amount] ,debit as [debit_amount],concat('ID-',cast(t.[Fiu_Batch_Id] as varchar)) batchid,p.Report_Id as reportsernum,personname,mobile, p.pan, t.account as accountnumber, address as commaddress, report_Type as reporttype, 
        --credit_amount credit,debit_amount debit, 
		email,'NA' AS groundsofsusp,'NA' AS sourceofalert
        from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[Combined_Transaction] t
        inner join cte3 p on p.reportid=t.reportid and concat('A-',p.accountnumber)=t.account AND LEFT(t.fiu_Batch_id,6)=202503
		)

    select distinct  p.batchid, reportsernum,personname,concat('M-',mobile)mobile, p.pan, p.accountnumber,  commaddress, p.reporttype, 
    trim(upper(re.name)) as entityname, 
    coalesce([oldREId],[newREId]) as fiureid,
    credit_amount credit,debit_amount debit, email,'NA' AS groundsofsusp,'NA' AS sourceofalert,
    addressline1 as branchaddress , ah.title as relationTypeId,branch_state
    from  cte_trans p
    --inner join cte_trans t on p.reportid=t.reportid and p.accountnumber=t.accountnumber 
    inner join  [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_AccountDetail] ad on ad.reportid=p.reportId and concat('A-',ad.accountnumber) =p.accountnumber 
    inner join [DATAHUB_DB].[FIUMetaHub].[finnet_ReportingEntity] re (nolock) on p.[Re_Id] = re.id_
    left join [DATAHUB_DB].[FIUMetaHub].[finnet_AcHolderType] ah on ah.id_= p.relationTypeId
    left join [DATAHUB_DB].[FIUMetaHub].[finnet_ReBranch] reb on ad.branchCode=reb.branchId and reb.reid=p.Re_Id
	LEFT JOIN [SAL_DB].[dbo].[state_meta_final] (NOLOCK) sm  ON sm.[parentId] = reb.id_
    left join [DATAHUB_DB].[FIUMetaHub].[finnet_Address] adr on reb.id_ = adr.parentid 
    where parentclass='Rebranch' and branch_state='JAMMU AND KASHMIR'     '''
    #print(query)
    #break
    engine_=sqlalchemy.create_engine('mssql+pyodbc://SAL_User01:%s@172.16.22.25:1433/FINCORE_BRIDGE_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@123'))
    path_to_file = ''
    file_name = 'filename'
    query_run(query,engine_,path_to_file,file_name)